### The Problem
New cars prices have become more and more expensive due to taxes 
regularized by the government. This increase in cost pushes many 
customers to buy used cars. Thus, both customers and sellers need to 
know which features to look for when buying or selling used cars 
Data set link: https://www.kaggle.com/austinreese/craigslist-carstrucks-data

In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [3]:
vehi_df = pd.read_csv("vehicles.csv")

In [4]:
vehi_df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


#### List the dataset column names

In [5]:
vehi_df.columns # there are 26 features in the dataset

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

#### Getting some stastistical values and checking for missing values

In [6]:
vehi_df.describe()

,id,price,year,odometer,county,lat,long
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05,0.0,420331.000000,420331.000000
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04,NaN,38.493940,-94.748599
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05,NaN,5.841533,18.365462
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-84.122245,-159.827728
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04,NaN,34.601900,-111.939847
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04,NaN,39.150100,-88.432600
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05,NaN,42.398900,-80.832039
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07,NaN,82.390818,173.885502


In [7]:
# there are missing values in columns: year, manifacturer, condition, model, cylinders, fuel,  odometer,
# title_status, transmission, VIN, drive, size, type, paint_color, image_url, description, state, lat, long, and posting_date
vehi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

### Data cleaning

In [8]:
vehi_df.isna().mean() # checking the percentage of null values in each column

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
condition       0.407852
cylinders       0.416225
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
VIN             0.377254
drive           0.305863
size            0.717675
type            0.217527
paint_color     0.305011
image_url       0.000159
description     0.000164
county          1.000000
state           0.000000
lat             0.015342
long            0.015342
posting_date    0.000159
dtype: float64

In [9]:
vehi_df.columns[vehi_df.isna().mean() >0.25]

Index(['condition', 'cylinders', 'VIN', 'drive', 'size', 'paint_color',
       'county'],
      dtype='object')

In [10]:
# Let us drop any column name that has OVER 25% null value
# any column names with true value will be drop
null_columns=vehi_df.columns[vehi_df.isna().mean() >0.25]

In [11]:
vehi_df=vehi_df.drop(null_columns, axis=1)

In [12]:
vehi_df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,image_url,description,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


#### Get rid of unwanted columns

In [13]:
#unwanted_column_names=['id', 'url', 'region_url','manufacturer','image_url', 'odometer','description', 'posting_date']
unwanted_column_names=['id', 'url', 'region_url','manufacturer','image_url', 'type', 'odometer', 'description', 'posting_date']

In [14]:
vehi_df=vehi_df.drop(unwanted_column_names, axis=1)

In [15]:
vehi_df

,region,price,year,model,fuel,title_status,transmission,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,maxima s sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,s60 t5 momentum sedan 4d,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,xt4 sport suv 4d,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,es 350 sedan 4d,gas,clean,other,wy,33.786500,-84.445400


#### Let us take care of categorical data. Check the number of unique values

In [16]:
{column: len(vehi_df[column].unique()) for column in vehi_df.columns if vehi_df.dtypes[column]=='object'}

{'region': 404,
 'model': 29668,
 'fuel': 6,
 'title_status': 7,
 'transmission': 4,
 'state': 51}

In [17]:
# It is not a good idea to add 29668 values in model column because it will increase the training time. So, we have to drop
# the model column
vehi_df= vehi_df.drop('model', axis=1)
vehi_df

,region,price,year,fuel,title_status,transmission,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,gas,clean,other,wy,33.786500,-84.445400


In [18]:
# if not working, get ride of
vehi_df.dropna() # drop rows where elements are missing

,region,price,year,fuel,title_status,transmission,state,lat,long
27,auburn,33590,2014.0,gas,clean,other,al,32.590000,-85.480000
28,auburn,22590,2010.0,gas,clean,other,al,32.590000,-85.480000
29,auburn,39590,2020.0,gas,clean,other,al,32.590000,-85.480000
30,auburn,30990,2017.0,gas,clean,other,al,32.590000,-85.480000
31,auburn,15000,2013.0,gas,clean,automatic,al,32.592000,-85.518900
...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,gas,clean,other,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,gas,clean,other,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,diesel,clean,other,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,gas,clean,other,wy,33.786500,-84.445400


In [19]:
def vehi_onehot_encode(df, columns, prefixes):
    df= df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df= pd.concat([df, dummies], axis=1)
        df= df.drop(column, axis=1)
        return df

In [20]:
vehi_df= vehi_onehot_encode(
    vehi_df,
    ['region', 'fuel','title_status', 'transmission', 'state'],
     ['reg', 'fuel', 'title', 'transmi', 'state']
)

In [21]:
vehi_df

,price,year,fuel,title_status,transmission,state,lat,long,reg_SF bay area,reg_abilene,...,reg_winchester,reg_winston-salem,reg_worcester / central MA,reg_wyoming,reg_yakima,reg_york,reg_youngstown,reg_yuba-sutter,reg_yuma,reg_zanesville / cambridge
0,6000,NaN,NaN,NaN,NaN,az,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11900,NaN,NaN,NaN,NaN,ar,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21000,NaN,NaN,NaN,NaN,fl,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,NaN,NaN,NaN,NaN,ma,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4900,NaN,NaN,NaN,NaN,nc,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,2019.0,gas,clean,other,wy,33.786500,-84.445400,0,0,...,0,0,0,1,0,0,0,0,0,0
426876,30590,2020.0,gas,clean,other,wy,33.786500,-84.445400,0,0,...,0,0,0,1,0,0,0,0,0,0
426877,34990,2020.0,diesel,clean,other,wy,33.779214,-84.411811,0,0,...,0,0,0,1,0,0,0,0,0,0
426878,28990,2018.0,gas,clean,other,wy,33.786500,-84.445400,0,0,...,0,0,0,1,0,0,0,0,0,0


In [22]:
vehi_df = vehi_df.drop(['year', 'fuel', 'title_status', 'transmission', 'state', 'lat', 'long'], axis=1)

In [23]:
for column in vehi_df.columns:
    vehi_df[column] = vehi_df[column].fillna(vehi_df[column].mean())

In [24]:
vehi_df.fillna(vehi_df.mean())

,price,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,reg_altoona-johnstown,reg_amarillo,reg_ames,reg_anchorage / mat-su,...,reg_winchester,reg_winston-salem,reg_worcester / central MA,reg_wyoming,reg_yakima,reg_york,reg_youngstown,reg_yuba-sutter,reg_yuma,reg_zanesville / cambridge
0,6000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,4900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,23590,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
426876,30590,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
426877,34990,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
426878,28990,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [25]:
# checking if there is still any missing values
vehi_df.isna().sum()

price                         0
reg_SF bay area               0
reg_abilene                   0
reg_akron / canton            0
reg_albany                    0
                             ..
reg_york                      0
reg_youngstown                0
reg_yuba-sutter               0
reg_yuma                      0
reg_zanesville / cambridge    0
Length: 405, dtype: int64

### Split and scale the vehi_df

In [26]:
y = vehi_df.loc[:,'price']
X = vehi_df.drop('price', axis=1)

In [27]:
# Create a scaler column
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

### Training

In [29]:
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
lin_y_pred = lin_model.predict(X_test)

In [30]:
lgb_model = lgb.LGBMRegressor(
    boosting_type = 'gbdt',
    num_leaves = 31,
    numb_estimators =100,
    reg_lambda= 1.0)
lgb_model.fit(X_train, y_train)
lgb_y_pred = lgb_model.predict(X_test)


[LightGBM] [Warning] Unknown parameter: numb_estimators


In [31]:
rng = np.random.RandomState(0)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
ridge_y_pred = ridge_model.predict(X_test)

In [32]:
from sklearn.linear_model import LassoCV
lasso_model = LassoCV(cv=5, random_state=0)
lasso_model.fit(X_train, y_train)
lasso_y_pred = lasso_model.predict(X_test)

In [33]:
from sklearn.ensemble import RandomForestRegressor
random_model = RandomForestRegressor(max_depth=2, random_state=0)
random_model.fit(X_train, y_train)
random_y_pred = random_model.predict(X_test)

In [34]:
#from sklearn.neighbors import KNeighborsRegressor
#knn_model = KNeighborsRegressor(n_neighbors=2)
#knn_model.fit(X_train, y_train)
#knn_y_pred = knn_model.predict(X_test)

### bagging classifier I tried to use ran over 4 hours so I decided to comment it out. I added this model to respond to a question asked during the discussion board on canvas

In [35]:
#Adding bagging classifier since I was asked in duscussions...
#bagging_model = BaggingClassifier(base_estimator=SVC(),n_estimators=10, random_state=0).fit(X_train, y_train)
#bagging_y_pred = bagging_model.predict(X_test)

In [36]:
lin_loss = np.sqrt(mean_squared_error(y_test, lin_y_pred))
lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_pred))
ridge_loss = np.sqrt(mean_squared_error(y_test, ridge_y_pred))
lasso_loss = np.sqrt(mean_squared_error(y_test, lasso_y_pred))
random_loss = np.sqrt(mean_squared_error(y_test, random_y_pred))
#knn_loss = np.sqrt(mean_squared_error(y_test, knn_y_pred))

#bagging_loss = np.sqrt(mean_squared_error(y_test, bagging_y_pred))

In [37]:
print("Linear Regression RMSE: ", lin_loss)
print("Gradient Boosted RMSE: ", lgb_loss)
print("Ridge RMSE: ", ridge_loss)
print("Lasso RMSE: ", lasso_loss)
print("Random Forest RMSE: ", random_loss)
#print("K Nearest Neighbors RMSE: ", knn_loss)


Linear Regression RMSE:  16158074.088270806
Gradient Boosted RMSE:  16158068.566060672
Ridge RMSE:  16158075.759365093
Lasso RMSE:  16154909.664497852
Random Forest RMSE:  16156951.308060637


In [38]:
print("Linear Regression R^2 Score: ", lin_model.score(X_test, y_test))
print("Gradien Booster R^2 Score: ", lgb_model.score(X_test, y_test))
print("Ridge  R^2 Score: ", ridge_model.score(X_test, y_test))
print("Lasso  R^2 Score: ", lasso_model.score(X_test, y_test))
print("Random Forest  R^2 Score: ", random_model.score(X_test, y_test))
#print("K Nearest Neighbors  R^2 Score: ", knn_model.score(X_test, y_test))


Linear Regression R^2 Score:  -0.00039834670026328567
Gradien Booster R^2 Score:  -0.00039766290479170685
Ridge  R^2 Score:  -0.00039855362592033927
Lasso  R^2 Score:  -6.545758267195723e-06
Random Forest  R^2 Score:  -0.0002593216611617155


### For the future, I need to evaluates the feature importance and run more models for better performance. I also need to plot some data for visualisation purposes.